# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core


In [ ]:
# | hide
from nbdev.showdoc import *


In [ ]:
# | export

import reader
from typing import *
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, features="lxml")
    return s.text


In [ ]:
# | export

class Feed:
    """RSS feed class 
    """

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)


In [ ]:
# | export

class PyNewsReader:
    def __init__(self, dbpath=None, feeds=List[Feed]):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self._reader = reader.make_reader(
            "db.sqlite", plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        self._reader.enable_search()
        self._feed_names = {}

    def _print_entries(self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = Text(
                        "Date: " + e.published.isoformat()[:10], justify="center"
                    )
                else:
                    published_date = Text("Date: Unknown", justify="center")
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = self._get_feed_title(e.original_feed_url)
                print(
                    Panel(
                        published_date
                        + "\n\n"
                        + Text(strip_html(e.summary) + "\n", justify=None),
                        title=f"[link={e.link}]{e.title}[/link]",
                        subtitle=feed_title,
                    )
                )
                print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed

        Args:
            url (str): URL of pynewsreader feed

        Returns:
            str: Display title
        """
        if url in self._feed_names:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()

    def _get_entries(self, important: bool = None, read: Union[None, bool] = None, limit: int = 10):
        """Get entries in reader.Entry format

        Args:
            read (reader.Entry.read, optional): Filter on `read` status (None, True, False). Defaults to None.
            limit (int, optional): Number of entries to return. Defaults to 10.

        Returns:
            List[reader.Entry]: List of entries
        """
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def add_feed(self, feed: Feed):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        self._reader.add_feed(feed.url, exist_ok=True)
        if feed.name:
            self._feed_names[feed.url] = feed.name

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def show(self, limit: int = 5, read: bool = None, important: bool = None, mark_as_read: bool = False):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(self._get_entries(
            read=read, important=important, limit=limit*2), limit=limit, mark_as_read=mark_as_read)

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries([self._search_to_entry(i) for i in self._reader.search_entries(
            query)], mark_as_read=mark_as_read, limit=limit)

    def mark_important(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_important(entry)

    def mark_unimportant(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_unimportant(entry)

        
    def tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Tag an entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
            tag_value (Dict, optional): Value of tag. Defaults to None.
        """
        reader.Reader.set_tag(entry, tag_key, tag_value)


In [ ]:
r = PyNewsReader()


## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url='https://ricochet.media/en/feed', name="Richochet Media"),
    Feed(url='https://thetyee.ca/rss2.xml', name="The Tyee"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
         name="Toronto Star | Top Stories"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
         name="Toronto Star | Investigations"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
         name="Toronto Star | Editorials"),
    Feed(url="https://www.macleans.ca/feed/",
         name="Macleans")
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))


## List Feeds

In [ ]:
r.feeds()


['Richochet Media',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Update Feeds

In [ ]:
r.update()


## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(read=False, limit=5, mark_as_read=True)


╭─────────── ]8;id=436695;https://www.cbc.ca/news/business/canadians-taxes-filing-money-owed-1.6688986?cmp=rss\As many as 1 in 10 Canadians don't file their taxes — and Ottawa owes many of them money]8;;\ ────────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│    Canadians who don't file their income tax returns are sometimes shocked to find out how much money they're   │
│   owed by the government for years of missed benefits, says the head of a non-profit organization working to    │
│                                build financial literacy among low-income people.                                │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭─────────────────── ]8;id=890570;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\Why burning plasma could be the next milestone in nuclear fusion research]8;;\ ───────────────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│  Bob McDonald's blog: A laser fusion experiment in the U.S. produced more energy out than went into driving it, │
│   which is an important step. But practical fusion power will likely come from a different technology and is    │
│                                             still many years away.                                              │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭──────── ]8;id=359665;https://www.cbc.ca/news/canada/ottawa/covid19-flu-rsv-ottawa-levels-december-2022-1.6688707?cmp=rss\COVID activity increasing, other respiratory viruses and influenza stable in latest OPH update]8;;\ ─────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│  Across all indicators, influenza and other respiratory virus activity are stable compared to last week, while  │
│                                        COVID-19 activity has increased.                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | Ottawa News ───────────────────────────────────────────────╯

╭──────────────── ]8;id=21684;https://www.thestar.com/news/gta/2022/12/16/hydro-one-says-23k-ontario-customers-are-still-without-power-after-winter-storm.html\Hydro One says 23K Ontario customers are still without power after winter storm]8;;\ ────────────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│ Crews have already restored power for about 100,000 customers but thousands may remain in the dark until closer │
│                                            to the end of next week.                                             │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╯

╭──────────────────────── ]8;id=5669;https://www.thestar.com/podcasts/thismatters/2022/12/16/scrooged-the-scams-of-christmas-present-a-true-rhyme-podcast.html\Scrooged: The scams of Christmas present (a true rhyme podcast)]8;;\ ────────────────────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│ TikTok creator Nichelle Laus and podcaster Stephen Lautens share holiday scams involving taxis and gift cards.  │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╯

## Search Entries

In [ ]:
r.search("fusion")


╭─────────────────── ]8;id=633694;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\Why burning plasma could be the next milestone in nuclear fusion research]8;;\ ───────────────────╮
│                                                Date: 2022-12-16                                                 │
│                                                                                                                 │
│  Bob McDonald's blog: A laser fusion experiment in the U.S. produced more energy out than went into driving it, │
│   which is an important step. But practical fusion power will likely come from a different technology and is    │
│                                             still many years away.                                              │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭─────────────────────────── ]8;id=322885;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\Fusion energy 'breakthrough' revealed by U.S. scientists]8;;\ ────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│      U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the      │
│                 decades-long quest to harness fusion, the energy that powers the sun and stars.                 │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────────────────── ]8;id=4051;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\What is nuclear fusion and why is it such a big deal?]8;;\ ─────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  The U.S. Department of Energy on Tuesday announced a breakthrough in nuclear fusion, a way of producing clean  │
│ energy that scientists have been working on since the 1940s. So what exactly is it, why is it so important and  │
│                                        what does it mean for the future?                                        │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭────────────── ]8;id=245919;https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss\The cannabis industry has a plastic waste problem — but some are finding solutions]8;;\ ───────────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│    In this week's issue of our environment newsletter, we look at the plastic waste produced in the cannabis    │
│      industry and what this week's announcement about nuclear fusion means for the future of clean energy.      │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | Canada News ───────────────────────────────────────────────╯

In [ ]:
# | hide

# To Do:
# * Command line interface
# * User interface
# * Tags


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()